In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
import cv2
import PIL
import os
import tqdm

In [2]:
video_file = "data/video/0gks6ceq4eQ.mp4"
caption_file = "data/captions/0gks6ceq4eQ.csv"

In [3]:
fps = 10                                # Number of frames per second

In [24]:
captions = pd.read_csv(caption_file)

for c in tqdm.trange(len(captions)):
    t, d = captions.iloc[c, [0, 1]]
    
    start_ft = t * fps // 1000
    end_ft = (t + d) * fps // 1000

    frame_ids = np.arange(start_ft, end_ft)

    prediction_file = "data/features/handpose/0gks6ceq4eQ/handpose_{t}_{d}.npy"
    
    if not os.path.exists(prediction_file.format(t=t, d=d)):
        continue
    
    if os.path.getsize(prediction_file.format(t=t, d=d)) == 278:
        continue
    
    landmarks = np.load(prediction_file.format(t=t, d=d))

    for f in frame_ids:
        i = f - start_ft
        
        fig, ax = plt.subplots(1, 2, figsize=(5, 3))

        lx, ly, rx, ry = landmarks[i, :21], landmarks[i, 21:42], landmarks[i, 42:63], landmarks[i, 63:]                
        ax[0].plot(lx, ly, "o")
        ax[0].set_xlim(-1, 1)
        ax[0].set_ylim(-1, 1)
        ax[0].get_xaxis().set_visible(False)
        ax[0].get_yaxis().set_visible(False)
        
        ax[1].plot(rx, ry, "o")
        ax[1].set_xlim(-1, 1)
        ax[1].set_ylim(-1, 1)
        ax[1].get_xaxis().set_visible(False)
        ax[1].get_yaxis().set_visible(False)
        
        fig.tight_layout()
        fig.savefig(f"tmp/hands_landmarks_1/img_{f}.png")
        plt.close()

100%|██████████| 332/332 [02:16<00:00,  2.43it/s]


In [25]:
w, h = (1280, 720)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer = cv2.VideoWriter("tmp/ground_truth.mp4", fourcc, fps, (w, h))

cap = cv2.VideoCapture(video_file)
    
while(cap.isOpened()):
    frame_exists, curr_frame = cap.read()
    if frame_exists:
        current_frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
        
        hands_landmarks_path = f"tmp/hands_landmarks_1/img_{current_frame_id}.png"
        
        if os.path.exists(hands_landmarks_path):
        
            hands_image = cv2.imread(hands_landmarks_path)
            
            x = np.zeros(curr_frame.shape, dtype=np.uint8)
            x[:hands_image.shape[0], :hands_image.shape[1], :] = hands_image

            curr_frame = cv2.addWeighted(curr_frame, 1, x, 0.6, 0)
            
        
        writer.write(curr_frame)
    else:
        break

writer.release() 